# Bin Packing Problem (BPP)

Given a set of $n$ items with weight $w_i$ for item $i=0,\ldots,n-1$, and an unlimited number of bins with identical capacities $c$ (all parameters are positive integers), assign each item to one bin so that the total weight of the items in each bin does not exceed $c$ and the number of bins used is a minimum.

Example application in shipping: The dispatcher of cargo has to decide how to assign a set of packages with given weights $w_i$ into a minimum number of containers. The only constraint is that the total weight packed in any container must not exceed the capacity $c$.

# Mathematical Model of BPP

$n$ = number of items, $UB=n$ = maximum number of bins

Variables:

$x_{ij}=1$ if item $i$ is assigned to bin $j$, $=0$ otherwise, $i=0,\ldots,n-1$, $j=0,\ldots,UB-1$ 

$y_j=1$ if bin $j$ is used, $=0$ otherwise, $j=0,\ldots,UB-1$

$\min \sum_{j=0}^{UB-1} y_j$ (number of bins used)

subject to:

$\sum_{j=0}^{UB-1} x_{ij}=1$ for $i=0,\ldots,n-1$ (item in exactly one bin)

$\sum_{i=0}^{n-1} w_i x_{ij} \leq c y_j$ for $j=0,\ldots,UB-1$ (total weight in each bin; also, linking $x_{ij}$ with $y_j$)

$x_{ij}\in\{0,1\}$ for $i=0,\ldots,n-1$, $j=0,\ldots,UB-1$

$y_j\in\{0,1\}$ for $j=0,\ldots,UB-1$

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

#import MATLAB data
column_name = ["Day1"] 
myFrame = pd.read_csv("dataset.csv", header =None, names = column_name)
dataset =myFrame.Day1.to_list()

bpp_data_set = [{'instance_name': 'u_60_0',    'c':  510, 'w': dataset}]
# bpp_data_set = [
#     {'instance_name': 'u_60_0',    'c':  150, 'w': [98, 93, 92, 90, 86, 85, 84, 84, 82, 82, 80, 79, 74, 74, 73, 72, 70, 69, 69, 67, 64, 62, 59, 57, 57, 57, 57, 55, 55, 50, 49, 49, 46, 46, 45, 44, 42, 42, 42, 41, 39, 38, 38, 38, 36, 36, 35, 33, 33, 33, 32, 30, 30, 30, 29, 28, 27, 27, 25, 23]},
#     {'instance_name': 't_60_0',    'c': 1000, 'w': [495, 474, 473, 472, 466, 450, 445, 444, 439, 430, 419, 414, 410, 395, 372, 370, 366, 366, 366, 363, 361, 357, 355, 351, 350, 350, 347, 320, 315, 307, 303, 299, 298, 298, 292, 288, 287, 283, 275, 275, 274, 273, 273, 272, 272, 271, 269, 269, 268, 263, 262, 261, 259, 258, 255, 254, 252, 252, 252, 251]},
#     {'instance_name': 'u_120_0',   'c':  150, 'w': [98, 98, 98, 96, 96, 94, 93, 93, 92, 91, 91, 90, 87, 86, 85, 85, 84, 84, 84, 84, 84, 83, 83, 82, 82, 81, 80, 80, 80, 79, 79, 78, 78, 78, 78, 76, 74, 74, 73, 73, 73, 73, 72, 71, 70, 70, 70, 69, 69, 69, 67, 66, 64, 62, 62, 60, 60, 59, 58, 58, 58, 57, 57, 57, 57, 55, 55, 55, 50, 49, 49, 49, 47, 46, 46, 45, 45, 44, 44, 43, 43, 43, 43, 42, 42, 42, 42, 42, 41, 41, 41, 39, 39, 38, 38, 38, 37, 36, 36, 36, 35, 33, 33, 33, 32, 32, 30, 30, 30, 29, 28, 27, 27, 26, 25, 25, 24, 23, 23, 20]},
#     {'instance_name': 'hard28_27', 'c': 1000, 'w': [794, 793, 792, 790, 788, 787, 784, 780, 779, 776, 774, 772, 770, 767, 762, 758, 756, 756, 753, 750, 747, 740, 740, 729, 728, 724, 719, 719, 710, 702, 702, 693, 690, 686, 682, 669, 662, 656, 655, 653, 641, 637, 634, 633, 633, 631, 628, 626, 625, 621, 610, 605, 600, 590, 583, 579, 578, 578, 568, 567, 563, 562, 562, 559, 552, 549, 548, 546, 546, 542, 540, 533, 529, 524, 522, 521, 521, 511, 500, 499, 492, 492, 489, 486, 483, 466, 466, 465, 463, 455, 451, 445, 445, 431, 426, 424, 411, 410, 405, 404, 388, 379, 379, 366, 364, 362, 361, 350, 348, 345, 344, 335, 327, 327, 323, 321, 319, 319, 315, 313, 312, 300, 296, 286, 285, 280, 277, 269, 267, 266, 262, 261, 243, 243, 234, 232, 230, 228, 225, 219, 217, 215, 214, 208, 207, 202, 199, 195, 193, 186, 186, 183, 180, 178, 176, 174, 173, 172, 172, 168, 163, 161, 159, 158, 150, 150, 148, 140, 138, 133, 126, 117, 116, 115, 111, 109, 106, 100, 97, 94, 92, 90, 84, 72, 72, 71, 71, 66, 53, 51, 49, 41, 40, 31, 28, 10, 7, 5, 3, 1]},
#     {'instance_name': 'u_250_0',   'c':  150, 'w': [100, 100, 100, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 96, 96, 95, 95, 95, 94, 94, 93, 93, 92, 92, 92, 91, 91, 90, 90, 90, 88, 88, 87, 86, 85, 85, 85, 84, 84, 84, 84, 84, 83, 83, 82, 82, 82, 81, 81, 81, 81, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 76, 76, 75, 75, 74, 74, 74, 73, 73, 73, 73, 72, 72, 72, 71, 71, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 68, 67, 67, 67, 67, 67, 66, 66, 66, 65, 65, 64, 64, 62, 62, 62, 61, 61, 60, 60, 60, 60, 60, 60, 59, 59, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 55, 55, 55, 55, 55, 53, 53, 53, 53, 53, 53, 52, 52, 50, 50, 49, 49, 49, 49, 49, 48, 48, 47, 47, 47, 47, 46, 46, 46, 46, 45, 45, 45, 45, 45, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 37, 37, 36, 36, 36, 36, 36, 36, 35, 35, 33, 33, 33, 33, 32, 32, 32, 32, 30, 30, 30, 30, 30, 29, 29, 29, 28, 27, 27, 27, 27, 27, 26, 25, 25, 25, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 22, 20, 20, 20, 20]},
#     {'instance_name': 'u_500_0',   'c':  150, 'w': [100, 100, 100, 100, 100, 100, 99, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 97, 97, 97, 97, 97, 97, 96, 96, 96, 96, 96, 96, 95, 95, 95, 95, 95, 95, 95, 94, 94, 94, 94, 93, 93, 92, 92, 92, 92, 92, 92, 91, 91, 91, 91, 91, 91, 91, 90, 90, 90, 90, 90, 90, 90, 90, 90, 89, 89, 88, 88, 88, 88, 87, 87, 87, 86, 86, 86, 86, 85, 85, 85, 85, 85, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 83, 83, 83, 83, 82, 82, 82, 82, 82, 81, 81, 81, 81, 81, 81, 81, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 78, 78, 77, 76, 76, 76, 76, 76, 76, 75, 75, 75, 75, 75, 74, 74, 74, 74, 74, 74, 73, 73, 73, 73, 73, 73, 73, 73, 72, 72, 72, 71, 71, 71, 71, 71, 71, 70, 70, 70, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 68, 68, 68, 68, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, 66, 66, 66, 66, 66, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 63, 63, 63, 62, 62, 62, 62, 62, 62, 61, 61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 60, 60, 59, 59, 59, 59, 59, 58, 58, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 56, 56, 55, 55, 55, 55, 55, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 53, 53, 52, 52, 52, 51, 51, 50, 50, 50, 50, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 33, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32, 32, 32, 32, 31, 31, 31, 30, 30, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28, 28, 28, 27, 27, 27, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 21, 21, 21, 21, 20, 20, 20, 20, 20]},
#     {'instance_name': 'set_1_714', 'c':  150, 'w': [100, 100, 100, 100, 100, 100, 99, 99, 99, 99, 99, 99, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 96, 96, 95, 95, 95, 95, 95, 95, 95, 95, 94, 94, 94, 94, 94, 94, 94, 94, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 92, 92, 92, 91, 91, 91, 91, 91, 91, 91, 90, 90, 90, 90, 90, 90, 90, 90, 90, 89, 89, 89, 89, 89, 89, 89, 89, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 87, 87, 87, 87, 87, 87, 87, 87, 87, 86, 86, 86, 86, 86, 86, 86, 85, 85, 85, 85, 85, 85, 85, 85, 85, 84, 84, 84, 84, 84, 84, 84, 84, 84, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 82, 82, 82, 82, 82, 82, 82, 82, 81, 81, 81, 81, 81, 81, 81, 81, 80, 80, 80, 80, 80, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 78, 77, 77, 77, 77, 77, 77, 77, 76, 76, 76, 76, 76, 76, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 74, 74, 74, 74, 74, 74, 74, 74, 73, 73, 73, 73, 73, 73, 72, 72, 72, 72, 72, 72, 72, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 68, 68, 68, 68, 68, 68, 68, 68, 67, 67, 67, 67, 67, 67, 67, 66, 66, 66, 66, 66, 66, 66, 66, 66, 65, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 63, 63, 63, 63, 63, 63, 63, 62, 62, 62, 62, 62, 61, 61, 61, 60, 60, 60, 60, 59, 59, 59, 59, 59, 58, 58, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 57, 57, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 55, 55, 55, 55, 54, 54, 54, 54, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 51, 51, 51, 50, 50, 50, 50, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 46, 46, 46, 46, 45, 44, 44, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 41, 41, 40, 40, 40, 40, 40, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 33, 33, 33, 32, 32, 32, 32, 32, 32, 32, 32, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30]},
#     {'instance_name': 'set_2_388', 'c': 1000, 'w': [626, 625, 624, 622, 620, 620, 620, 619, 613, 611, 610, 609, 608, 606, 606, 604, 601, 601, 601, 600, 598, 598, 597, 591, 587, 586, 586, 586, 584, 584, 584, 584, 583, 583, 582, 582, 581, 581, 581, 579, 579, 579, 578, 578, 578, 576, 573, 570, 569, 567, 567, 565, 564, 562, 559, 559, 558, 557, 555, 553, 553, 550, 550, 547, 545, 544, 543, 542, 541, 541, 540, 540, 539, 539, 537, 536, 535, 533, 532, 531, 529, 528, 527, 527, 525, 524, 524, 523, 521, 520, 520, 518, 518, 518, 517, 517, 516, 516, 515, 514, 514, 512, 507, 506, 505, 505, 504, 503, 502, 502, 502, 501, 500, 499, 499, 497, 497, 496, 495, 495, 495, 494, 493, 491, 491, 487, 485, 484, 483, 482, 480, 479, 478, 475, 475, 475, 472, 471, 471, 469, 468, 467, 466, 465, 465, 463, 463, 462, 462, 462, 462, 461, 461, 461, 460, 458, 457, 457, 456, 454, 454, 452, 451, 447, 443, 443, 442, 439, 439, 439, 438, 437, 435, 434, 433, 431, 431, 428, 428, 428, 427, 427, 425, 425, 423, 421, 420, 419, 417, 416, 415, 412, 411, 411, 406, 405, 404, 401, 401, 400, 397, 397, 396, 395, 394, 394, 394, 393, 393, 390, 390, 388, 388, 386, 385, 383, 381, 378, 378, 377, 377, 376, 375, 375, 373, 372, 370, 369, 369, 367, 366, 365, 365, 364, 364, 363, 360, 359, 359, 358, 354, 353, 353, 353, 352, 350, 349, 348, 345, 345, 345, 344, 342, 342, 341, 340, 335, 333, 333, 332, 331, 331, 329, 328, 327, 326, 326, 325, 325, 322, 322, 321, 321, 321, 320, 318, 317, 317, 317, 317, 317, 317, 316, 315, 314, 313, 313, 312, 310, 308, 307, 307, 306, 306, 306, 302, 298, 296, 296, 295, 295, 295, 293, 293, 291, 289, 288, 287, 287, 286, 285, 285, 282, 281, 280, 275, 274, 274, 270, 269, 269, 268, 268, 266, 265, 265, 263, 263, 263, 263, 262, 261, 258, 257, 257, 257, 255, 253, 252, 250, 250, 246, 243, 243, 240, 240, 237, 237, 236, 234, 234, 233, 231, 230, 228, 227, 226, 226, 225, 225, 223, 221, 220, 220, 218, 217, 217, 216, 214, 212, 212, 211, 206, 206, 203, 203, 202, 202, 201, 201, 201, 201, 200, 194, 194, 194, 192, 191, 190, 186, 186, 183, 183, 174, 171, 167, 167, 167, 166, 163, 163, 162, 159, 158, 157, 156, 156, 151, 150, 148, 145, 145, 143, 142, 141, 137, 136, 132, 132, 131, 131, 129, 129, 128, 126, 126, 125, 125, 122, 121, 120, 119, 114, 113, 112, 111, 109, 109, 109, 109, 106, 105, 105, 102, 102, 100, 95, 95, 91, 91, 88, 88, 87, 84, 84, 82, 81, 80, 78, 76, 75, 75, 73, 73, 73, 72, 69, 69, 68, 67, 65, 65, 64, 64, 62, 61, 59, 57, 57, 53, 51, 51, 49, 49, 49, 49, 48, 47, 46, 45, 44, 43, 42, 42, 41, 39, 39, 38, 37, 35]},
#     {'instance_name': 'u_1000_0',  'c':  150, 'w': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 94, 94, 94, 94, 94, 94, 94, 94, 94, 93, 93, 93, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 89, 89, 89, 89, 89, 89, 89, 89, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 86, 86, 86, 86, 86, 86, 86, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 77, 77, 77, 77, 77, 76, 76, 76, 76, 76, 76, 76, 76, 76, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 74, 74, 74, 74, 74, 74, 74, 74, 74, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 68, 68, 68, 68, 68, 68, 68, 68, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 65, 65, 65, 65, 65, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 63, 63, 63, 63, 63, 63, 63, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 56, 56, 56, 56, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 54, 54, 54, 54, 54, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51, 51, 51, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28, 28, 28, 28, 28, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]},
# ];

def lb(c, w):
    return int(math.ceil(sum(w) / c))

df = pd.DataFrame(bpp_data_set, columns=['instance_name', 'c', 'w'])
df['n'] = df['w'].apply(len)
df['wmin'] = df['w'].apply(min)
df['wmax'] = df['w'].apply(max)
df['lb'] = df.apply(lambda x: lb(x['c'], x['w']), axis=1)
display(df)


,instance_name,c,w,n,wmin,wmax,lb
0,u_60_0,510,"[19, 2, 33, 310, 11, 11, 301, 53, 15, 14, 3, 2...",100,1,322,10


In [2]:
def model_bpp(c, w, UB=None, bin_for_item=None, LogToConsole=True, TimeLimit=30):
    n = len(w)
    LB = lb(c, w)
    if UB == None:
        UB = n
    if LogToConsole:
        print('c =', c, '| n =', n, '| LB =', LB, '| UB =', UB)
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = model.addVars(n, UB, vtype=GRB.BINARY)
    y = model.addVars(UB, vtype=GRB.BINARY)
    model.setObjective(quicksum(y[j] for j in range(UB)), GRB.MINIMIZE) # minimize the number of bins used
    model.addConstrs(quicksum(x[i,j] for j in range(UB)) == 1 for i in range(n)) # each item in exactly one bin
    model.addConstrs(quicksum(w[i] * x[i,j] for i in range(n)) <= c * y[j] for j in range(UB)) # limit total weight in each bin; also, link $x_{ij}$ with $y_j$
    
    if bin_for_item != None:
        for i in range(n):
            x[i, bin_for_item[i]].start = 1
    model.optimize()
    bin_for_item = [-1 for i in range(n)]
    for i in range(n):
        for j in range(UB):
            if x[i,j].X > 0.5:
                bin_for_item[i] = j
    return model.ObjVal, model.ObjBound, bin_for_item


bpp_data = bpp_data_set[0]
c, w = bpp_data['c'], bpp_data['w']
ObVal, ObBound, bin_for_item = model_bpp(c,w)




def genQ(bin4item):
    queues = []
    newarray=[[],[],[],[],[],[],[],[],[]]
    uniqueList = []
    unique_numbers = set(bin4item)
    for number in unique_numbers:
        uniqueList.append(number)
    
    for i in range(len(uniqueList)):
        queues.append([index for index, element in enumerate(bin_for_item) if element == uniqueList[i]])

    count = 0
    for i in queues:
        for j in i:
            newarray[count].append(bpp_data['w'][j])
        count += 1
        
    return uniqueList, queues, newarray


uniqueList, queues, newarray = genQ(bin_for_item)
print("Individual cars in queues: ", newarray)
meF = pd.DataFrame(newarray)
meF = meF.transpose()
print(meF) 



c = 510 | n = 100 | LB = 10 | UB = 100
Academic license - for non-commercial use only - expires 2021-06-16
Using license file /Users/jordankotler/gurobi.lic
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 200 rows, 10100 columns and 20100 nonzeros
Model fingerprint: 0x9058843f
Variable types: 0 continuous, 10100 integer (10100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 65.0000000
Presolve time: 0.08s
Presolved: 200 rows, 10100 columns, 20100 nonzeros
Variable types: 0 continuous, 10100 integer (10100 binary)

Root relaxation: objective 9.762745e+00, 497 iteration

IndexError: list index out of range

In [ ]:
def heur_FFD(c, w): # first-fit-decreasing heuristic
    n = len(w)
    order = sorted([i for i in range(n)], key=lambda i: w[i], reverse=True) # sort by decreasing weights
    bin_for_item = [-1 for i in range(n)]
    bin_space = []
    for i in order:
        for j in range(len(bin_space)): # pack in the first bin in which the item fits
            if w[i] <= bin_space[j]:
                bin_for_item[i] = j
                bin_space[j] -= w[i]
                break
        if bin_for_item[i] < 0: # if no bin can accomodate the item, open a new bin
            j = len(bin_space)
            bin_for_item[i] = j
            bin_space.append(c - w[i])
    n_bins = len(bin_space)
    return n_bins, bin_for_item

# genQ(bin_for_item)



In [ ]:
bpp_data = bpp_data_set[0]
c, w = bpp_data['c'], bpp_data['w']
n_bins, bin_for_item = heur_FFD(c, w)
print("Heur found a solution with n_bins =", n_bins)

firstRe, secondRe, thirdRe = genQ(bin_for_item)
print(thirdRe)
heurFrame = pd.DataFrame(thirdRe)
heurFrame = heurFrame.transpose()
results = sum(bpp_data['w'])
kms = results/1.44
print("The total times of all cars is", results, "and the total Kms possible is", kms)

print(heurFrame) 




In [ ]:
for idx, bpp_data in enumerate(bpp_data_set):
    c, w = bpp_data['c'], bpp_data['w']
    n_bins, bin_for_item = heur_FFD(c, w)
    print(idx, n_bins)

In [ ]:
for idx, bpp_data in enumerate(bpp_data_set):
    c, w = bpp_data['c'], bpp_data['w']
    n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, LogToConsole=False, TimeLimit=5)
    print(idx, n_bins, n_bins_lb)

In [ ]:
from time import time

for idx, bpp_data in enumerate(bpp_data_set):
    t_start = time()
    c, w = bpp_data['c'], bpp_data['w']
    n_bins, bin_for_item = heur_FFD(c, w)
    n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, n_bins, bin_for_item, LogToConsole=False, TimeLimit=10)
    t_end = time()
    print(idx, n_bins, n_bins_lb, t_end - t_start)